<a href="https://colab.research.google.com/github/myielin/dataMiningUTFPR/blob/master/geneFeatures/ZosteraMarinaRNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento com MathFeature


[Repositório MathFeature](https://bonidia.github.io/MathFeature/)
<br>
[Repo do notebook](https://github.com/myielin/dataMiningUTFPR/geneFeatures/)


### MathFeature Preprocssing:

Os scripts do framework foram executados localmente nas sequências fasta 

(linha de comando, na pasta MathFeature/preprocessing)


```
python3 count_sequences.py -i ../[PATH]/Zmarina-lncRNA.fasta
```



- número de sequências em mRNA: 21483
- número de sequências em lncRNA: 1393

Por haver uma diferença muito grande nas quantidades de sequências, o primeiro arquivo deve ser reduzido proporcionalmente ao segundo.
<br>
Para isso, no próximo exemplo é criado um novo arquivo fasta com a amostra necessária utilizando o script sampling.py



```
python3 sampling.py -i ../[PATH]/gencode.v42.transcripts.fa -o ../[PATH]/samp-mRNA.fa -p 1393
```

Em seguida, são gerados mais dois arquivos com o pré-processamento do mRNA e lncRNA, onde são removidos os ruídos referentes a caracteres que não correspondem a nucleotídeos:



```
python3 preprocessing.py -i ../[PATH]/samp-mRNA.fa -o ../[PATH]/prep-mRNA.fa
```


```
python3 preprocessing.py -i ../[PATH]/samp-mRNA.fa -o ../[PATH]/prep-lncRNA.fa
```







### Mathfeature Methods:

No exemplo serão utilizados os métodos kmer, transformada de Fourier e entropia de Shannon. Cada método gera um arquivo de csv com base nas informações processadas em fasta

Kmer Args:
- l: label - corresponde a como a sequencia de entrada será representada. Aqui será usado 0 para RNA não codificante, pois a busca será por RNA mensageiro (1)
- t: tipo de extração
- seq: RNA (1) ou DNA 

```
python3 ExtractionTechniques.py -i ../[PATH]/prep-lncRNA.fa -o ../[PATH]/kmer-lncRNA.csv -l 0 -t kmer -seq 1
```

```
python3 ExtractionTechniques.py -i ../[PATH]/prep-mRNA.fa -o ../[PATH]/kmer-mRNA.csv -l 1 -t kmer -seq 1
```





Shannon Entropy Args:
- k: tamanho do kmer que é utilizado nas medições de entropia 
- e: tipo de entropia (Shannon ou Tsallis)

```
python3 EntropyClass.py -i ../[PATH]/prep-lncRNA.fa -o ../[PATH]/shannon-lncRNA.csv -l 0 -k 3 -e Shannon
```

```
python3 EntropyClass.py -i ../[PATH]/prep-mRNA.fa -o ../[PATH]/shannon-mRNA.csv -l 1 -k 3 -e Shannon
```





Transformação de Fourier 


```
python3 FourierClass.py -i ../[PATH]/prep-lncRNA.fa -o ../[PATH]/fourier-lncRNA.csv -l 0 -r 3
```

```
python3 FourierClass.py -i ../[PATH]/prep-mRNA.fa -o ../[PATH]/fourier-mRNA.csv -l 1 -r 3
```


Os arquivos csv das diferentes classes de RNA foram então concatenados, gerando os arquivos específicos para lncRNA e mRNA contendo as 3 técnicas.


(na pasta onde estão localizados os arquivos csv)
```
python3 MathFeature/preprocessing/concatenate.py -n 3 -o lncRNA.csv

```
(a ordem de concatenação dos conjuntos foi: kmer, fourier, shannon)



### pré-processamento nos conjuntos gerados

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts

In [ ]:
mrna = pd.read_csv("https://raw.githubusercontent.com/myielin/dataMiningUTFPR/master/geneFeatures/mRNA.csv")
lrna = pd.read_csv("https://raw.githubusercontent.com/myielin/dataMiningUTFPR/master/geneFeatures/lncRNA.csv")

Alguns valores gerados foram NaN, e serão preenchidos utilizando o método ffill ou dropados <br> <br>
Os conjuntos de mRNA e lncRNA serão concatenados em uma base geral de rna

In [ ]:
lrna.shape

(1393, 108)

In [ ]:
rna = pd.concat([lrna, mrna])

Os demais valores categóricos precisam ser transformados em numéricos para então serem divididos os conjuntos de treino e teste

In [ ]:
# dropando a linha onde a concatenação gera strings com os nomes dos atributos
rna.drop(1393, axis=0, inplace=True)

# rna.fillna(method="ffill", inplace=True)
rna.dropna(inplace=True)

rna.drop('nameseq', axis=1, inplace = True)

# valores do tipo int e str estão misturados nas variáveis, então devem ser convertidos em numéricos
for i in (rna.columns):
  rna[i] = rna[i].apply(lambda k: float(k))

In [ ]:
# separando os conjuntos x e y 
y = rna['label']
x = rna.drop('label', axis=1)

In [ ]:
x_train, x_test, y_train, y_test = tts(x, y, test_size = 0.25)

# Modelos preditivos

In [ ]:
from sklearn.metrics import (recall_score,
                             accuracy_score,
                             precision_score,
                             f1_score)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier as Knn

In [ ]:
dtc = DecisionTreeClassifier()

dtc.fit(x_train, y_train)
pred = dtc.predict(x_test)

acuracidade = round(accuracy_score(y_test, pred)*100, 2)
print(acuracidade)

100.0


In [ ]:
knn = Knn(n_neighbors=3)

knn.fit(x_train, y_train)
pred = knn.predict(x_test)

precisao = round(precision_score(y_test, pred)*100, 2)
print(precisao)

99.44


In [ ]:
nb = GaussianNB()

nb.fit(x_train, y_train)
pred = nb.predict(x_test)

precisao = round(accuracy_score(y_test, pred)*100, 2)
print(precisao)

99.71
